In [1]:
import shioaji_login
import shioaji as sj

In [2]:
api = shioaji_login.shioaji_login()

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.
<SecurityType.Future: 'FUT'> fetch done.
True


In [3]:
contract_0050 = api.Contracts.Stocks["0050"]
contract_0050

<SecurityType.Stock: 'STK'> fetch done.


Stock(exchange=<Exchange.TSE: 'TSE'>, code='0050', symbol='TSE0050', name='元大台灣50', category='00', unit=1000, limit_up=155.65, limit_down=127.35, reference=141.5, update_date='2021/12/14', margin_trading_balance=6395, day_trade=<DayTrade.Yes: 'Yes'>)

In [4]:
ticks = api.ticks(contract_0050, "2012-12-06")
ticks

### 不能用，誰知道為什麼呢？

<SecurityType.Option: 'OPT'> fetch done.


Ticks(ts=[], close=[], volume=[], bid_price=[], bid_volume=[], ask_price=[], ask_volume=[])

In [5]:
import pandas as pd
kbars = api.kbars(contract_0050, start="2021-12-07", end="2021-12-07")
kbars_df = pd.DataFrame({**kbars})
kbars_df.ts = pd.to_datetime(kbars_df.ts)
#kbars_df.head()
kbars_df.tail()

,Close,Open,Low,ts,High,Volume,Amount
211,141.45,141.45,141.45,2021-12-07 13:22:00,141.50,11,1556400.0
212,141.60,141.55,141.45,2021-12-07 13:23:00,141.60,31,4388050.0
213,141.65,141.60,141.55,2021-12-07 13:24:00,141.65,17,2407550.0
214,141.65,141.65,141.60,2021-12-07 13:25:00,141.70,12,1699800.0
215,141.80,141.80,141.80,2021-12-07 13:30:00,141.80,75,10635000.0


In [6]:
contract_txf = api.Contracts.Futures['MXFL1']
contract_txf

Future(code='MXFL1', symbol='MXF202112', name='小型臺指12', category='MXF', delivery_month='202112', delivery_date='2021/12/15', underlying_kind='I', unit=1, limit_up=19528.0, limit_down=15978.0, reference=17753.0, update_date='2021/12/14')

In [20]:
ticks = api.ticks(contract_txf, "2012-12-06")
ticks

Ticks(ts=[], close=[], volume=[], bid_price=[], bid_volume=[], ask_price=[], ask_volume=[])

這可以接受最新的kbar

In [7]:
kbars = api.kbars(contract_txf, start="2021-12-07", end="2021-12-07")
kbars_df = pd.DataFrame({**kbars})
kbars_df.ts = pd.to_datetime(kbars_df.ts)
#kbars_df.head()
kbars_df.tail()

,Close,Open,Low,ts,High,Volume,Amount
1135,17909.0,17908.0,17906.0,2021-12-07 23:56:00,17909.0,71,1271404.0
1136,17907.0,17908.0,17906.0,2021-12-07 23:57:00,17909.0,62,1110242.0
1137,17907.0,17907.0,17907.0,2021-12-07 23:58:00,17909.0,49,877509.0
1138,17908.0,17907.0,17907.0,2021-12-07 23:59:00,17909.0,36,644680.0
1139,17907.0,17908.0,17905.0,2021-12-08 00:00:00,17909.0,161,2883051.0


可以利用callback(?)修改class中的資料然後獲得最新報價

In [8]:
class QuoteManager:
    
    def __init__(self):
        self.quote = ''
        
    def print_quote(self):
        print(f"My quote is: {self.quote}!")

q = QuoteManager()

In [9]:
# callback 
@api.quote.on_quote
def quote_callback(topic: str, quote: dict):
    #print(f"Topic: {topic}, Quote: {quote}")
    q.quote = quote
    q.print_quote()

In [10]:
# subscribe tick data
api.quote.subscribe(contract_txf, quote_type=sj.constant.QuoteType.Tick)

Response Code: 200 | Event Code: 16 | Info: L/*/MXFL1 | Event: Subscribe or Unsubscribe ok
My quote is: {'Amount': [17596.0], 'AmountSum': [2248459162.0], 'AvgPrice': [17639.954513], 'Close': [17596.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-157.0], 'DiffRate': [-0.884358], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17584.54, 'TickType': [1], 'Time': '12:37:24.591000', 'TradeAskVolSum': 90040, 'TradeBidVolSum': 95313, 'VolSum': [127464], 'Volume': [1]}!
My quote is: {'Amount': [17596.0], 'AmountSum': [2248476758.0], 'AvgPrice': [17639.954168], 'Close': [17596.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-157.0], 'DiffRate': [-0.884358], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17583.48, 'TickType': [2], 'Time': '12:37:25.326000', 'TradeAskVolSum': 90042, 'TradeBidVolSum': 95313, 'VolSum': [127465], 'Volume': [1]}!
My quote is: {'Amount': [17596.0], 'AmountSum': [22

My quote is: {'Amount': [35188.0], 'AmountSum': [2249004603.0], 'AvgPrice': [17639.943551], 'Close': [17594.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-159.0], 'DiffRate': [-0.895623], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17592.21, 'TickType': [2], 'Time': '12:37:31.746000', 'TradeAskVolSum': 90063, 'TradeBidVolSum': 95341, 'VolSum': [127495], 'Volume': [2]}!
My quote is: {'Amount': [17594.0], 'AmountSum': [2249022197.0], 'AvgPrice': [17639.94319], 'Close': [17594.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-159.0], 'DiffRate': [-0.895623], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17592.21, 'TickType': [2], 'Time': '12:37:31.777000', 'TradeAskVolSum': 90063, 'TradeBidVolSum': 95343, 'VolSum': [127496], 'Volume': [1]}!
My quote is: {'Amount': [17593.0], 'AmountSum': [2249039790.0], 'AvgPrice': [17639.942822], 'Close': [17593.0], 'Code': 'MXFL1', 'Date': '2021/

My quote is: {'Amount': [17592.0], 'AmountSum': [2249462019.0], 'AvgPrice': [17639.933964], 'Close': [17592.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-161.0], 'DiffRate': [-0.906889], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17592.21, 'TickType': [2], 'Time': '12:37:32.699000', 'TradeAskVolSum': 90082, 'TradeBidVolSum': 95368, 'VolSum': [127521], 'Volume': [1]}!
My quote is: {'Amount': [17592.0], 'AmountSum': [2249479611.0], 'AvgPrice': [17639.933588], 'Close': [17592.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-161.0], 'DiffRate': [-0.906889], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17592.21, 'TickType': [2], 'Time': '12:37:32.731000', 'TradeAskVolSum': 90083, 'TradeBidVolSum': 95369, 'VolSum': [127522], 'Volume': [1]}!
My quote is: {'Amount': [17592.0], 'AmountSum': [2249497203.0], 'AvgPrice': [17639.933212], 'Close': [17592.0], 'Code': 'MXFL1', 'Date': '2021

My quote is: {'Amount': [17594.0], 'AmountSum': [2249954601.0], 'AvgPrice': [17639.923488], 'Close': [17594.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-159.0], 'DiffRate': [-0.895623], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17592.21, 'TickType': [1], 'Time': '12:37:34.364000', 'TradeAskVolSum': 90104, 'TradeBidVolSum': 95395, 'VolSum': [127549], 'Volume': [1]}!
My quote is: {'Amount': [70368.0], 'AmountSum': [2250024969.0], 'AvgPrice': [17639.921985], 'Close': [17592.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-161.0], 'DiffRate': [-0.906889], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17588.2, 'TickType': [2], 'Time': '12:37:35.519000', 'TradeAskVolSum': 90109, 'TradeBidVolSum': 95395, 'VolSum': [127553], 'Volume': [4]}!
My quote is: {'Amount': [17592.0], 'AmountSum': [2250042561.0], 'AvgPrice': [17639.92161], 'Close': [17592.0], 'Code': 'MXFL1', 'Date': '2021/1

My quote is: {'Amount': [17592.0], 'AmountSum': [2250992607.0], 'AvgPrice': [17639.901942], 'Close': [17592.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-161.0], 'DiffRate': [-0.906889], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17581.87, 'TickType': [2], 'Time': '12:37:45.875000', 'TradeAskVolSum': 90125, 'TradeBidVolSum': 95440, 'VolSum': [127608], 'Volume': [1]}!
My quote is: {'Amount': [17592.0], 'AmountSum': [2251010199.0], 'AvgPrice': [17639.901567], 'Close': [17592.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-161.0], 'DiffRate': [-0.906889], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17581.87, 'TickType': [2], 'Time': '12:37:46.097000', 'TradeAskVolSum': 90126, 'TradeBidVolSum': 95441, 'VolSum': [127609], 'Volume': [1]}!
My quote is: {'Amount': [17593.0], 'AmountSum': [2251027792.0], 'AvgPrice': [17639.901199], 'Close': [17593.0], 'Code': 'MXFL1', 'Date': '2021

...

My quote is: {'Amount': [17598.0], 'AmountSum': [2312845303.0], 'AvgPrice': [17638.880607], 'Close': [17598.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-155.0], 'DiffRate': [-0.873092], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17592.69, 'TickType': [1], 'Time': '12:56:29.665000', 'TradeAskVolSum': 92613, 'TradeBidVolSum': 97858, 'VolSum': [131122], 'Volume': [1]}!
My quote is: {'Amount': [17597.0], 'AmountSum': [2312862900.0], 'AvgPrice': [17638.880288], 'Close': [17597.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-156.0], 'DiffRate': [-0.878725], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17593.14, 'TickType': [2], 'Time': '12:56:30.129000', 'TradeAskVolSum': 92615, 'TradeBidVolSum': 97858, 'VolSum': [131123], 'Volume': [1]}!
My quote is: {'Amount': [17597.0], 'AmountSum': [2312880497.0], 'AvgPrice': [17638.879969], 'Close': [17597.0], 'Code': 'MXFL1', 'Date': '2021

My quote is: {'Amount': [17600.0], 'AmountSum': [2313725256.0], 'AvgPrice': [17638.865429], 'Close': [17600.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-153.0], 'DiffRate': [-0.861826], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17593.14, 'TickType': [2], 'Time': '12:56:33.655000', 'TradeAskVolSum': 92659, 'TradeBidVolSum': 97878, 'VolSum': [131172], 'Volume': [1]}!
My quote is: {'Amount': [70404.0], 'AmountSum': [2313795660.0], 'AvgPrice': [17638.864274], 'Close': [17601.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-152.0], 'DiffRate': [-0.856193], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17593.14, 'TickType': [1], 'Time': '12:56:33.768000', 'TradeAskVolSum': 92659, 'TradeBidVolSum': 97883, 'VolSum': [131176], 'Volume': [4]}!
My quote is: {'Amount': [17601.0], 'AmountSum': [2313813261.0], 'AvgPrice': [17638.863985], 'Close': [17601.0], 'Code': 'MXFL1', 'Date': '2021

My quote is: {'Amount': [17601.0], 'AmountSum': [2314182884.0], 'AvgPrice': [17638.85794], 'Close': [17601.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-152.0], 'DiffRate': [-0.856193], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17591.24, 'TickType': [1], 'Time': '12:56:37.010000', 'TradeAskVolSum': 92678, 'TradeBidVolSum': 97900, 'VolSum': [131198], 'Volume': [1]}!
My quote is: {'Amount': [17601.0], 'AmountSum': [2314200485.0], 'AvgPrice': [17638.857651], 'Close': [17601.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-152.0], 'DiffRate': [-0.856193], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17591.24, 'TickType': [1], 'Time': '12:56:37.147000', 'TradeAskVolSum': 92679, 'TradeBidVolSum': 97901, 'VolSum': [131199], 'Volume': [1]}!
My quote is: {'Amount': [17600.0], 'AmountSum': [2314218085.0], 'AvgPrice': [17638.857355], 'Close': [17600.0], 'Code': 'MXFL1', 'Date': '2021/

My quote is: {'Amount': [17599.0], 'AmountSum': [2315097999.0], 'AvgPrice': [17638.841897], 'Close': [17599.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-154.0], 'DiffRate': [-0.867459], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17591.71, 'TickType': [2], 'Time': '12:56:51.179000', 'TradeAskVolSum': 92705, 'TradeBidVolSum': 97936, 'VolSum': [131250], 'Volume': [1]}!
My quote is: {'Amount': [70400.0], 'AmountSum': [2315168399.0], 'AvgPrice': [17638.840713], 'Close': [17600.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-153.0], 'DiffRate': [-0.861826], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17591.71, 'TickType': [1], 'Time': '12:56:53.978000', 'TradeAskVolSum': 92705, 'TradeBidVolSum': 97941, 'VolSum': [131254], 'Volume': [4]}!
My quote is: {'Amount': [17600.0], 'AmountSum': [2315185999.0], 'AvgPrice': [17638.840418], 'Close': [17600.0], 'Code': 'MXFL1', 'Date': '2021

My quote is: {'Amount': [17600.0], 'AmountSum': [2315661210.0], 'AvgPrice': [17638.832513], 'Close': [17600.0], 'Code': 'MXFL1', 'Date': '2021/12/14', 'DiffPrice': [-153.0], 'DiffRate': [-0.861826], 'DiffType': [4], 'High': [17711.0], 'Low': [17583.0], 'Open': 17684.0, 'TargetKindPrice': 17593.35, 'TickType': [0], 'Time': '12:57:07.944000', 'TradeAskVolSum': 92732, 'TradeBidVolSum': 97959, 'VolSum': [131282], 'Volume': [1]}!


In [51]:
# unsubscribe tick data
api.quote.unsubscribe(contract_txf, quote_type=sj.constant.QuoteType.Tick)

Response Code: 200 | Event Code: 16 | Info: L/*/MXFL1 | Event: Subscribe or Unsubscribe ok


In [39]:
print(MyClass.i)

L/TFE/MXFL1


In [30]:
api.quote.subscribe(contract_txf, quote_type=sj.constant.QuoteType.BidAsk)

Response Code: 200 | Event Code: 16 | Info: Q/*/MXFL1 | Event: Subscribe or Unsubscribe ok
Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17661.0, 17662.0, 17663.0, 17664.0, 17665.0], 'AskVolSum': 144, 'AskVolume': [12, 26, 25, 36, 45], 'BidPrice': [17660.0, 17659.0, 17658.0, 17657.0, 17656.0], 'BidVolSum': 129, 'BidVolume': [11, 25, 32, 31, 30], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [0, 0, 1, 0, -1], 'DiffAskVolSum': 0, 'DiffBidVol': [0, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17663.0, 'FirstDerivedAskVolume': 2, 'FirstDerivedBidPrice': 17657.0, 'FirstDerivedBidVolume': 10, 'TargetKindPrice': 17677.12, 'Time': '12:03:47.397000'}
Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17661.0, 17662.0, 17663.0, 17664.0, 17665.0], 'AskVolSum': 141, 'AskVolume': [9, 26, 25, 36, 45], 'BidPrice': [17660.0, 17659.0, 17658.0, 17657.0, 17656.0], 'BidVolSum': 130, 'BidVolume': [12, 25, 32, 31, 30], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [-3, 0, 0, 0, 0], 'DiffAskVolSum

Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17661.0, 17662.0, 17663.0, 17664.0, 17665.0], 'AskVolSum': 144, 'AskVolume': [11, 26, 26, 36, 45], 'BidPrice': [17660.0, 17659.0, 17658.0, 17657.0, 17656.0], 'BidVolSum': 134, 'BidVolume': [15, 25, 32, 30, 32], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [0, 0, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [0, 0, 0, 0, 2], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17663.0, 'FirstDerivedAskVolume': 2, 'FirstDerivedBidPrice': 17657.0, 'FirstDerivedBidVolume': 10, 'TargetKindPrice': 17677.08, 'Time': '12:03:50.522000'}
Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17661.0, 17662.0, 17663.0, 17664.0, 17665.0], 'AskVolSum': 145, 'AskVolume': [12, 26, 26, 36, 45], 'BidPrice': [17660.0, 17659.0, 17658.0, 17657.0, 17656.0], 'BidVolSum': 131, 'BidVolume': [12, 25, 32, 30, 32], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [1, 0, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [-3, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17663.0, '

Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17661.0, 17662.0, 17663.0, 17664.0, 17665.0], 'AskVolSum': 153, 'AskVolume': [12, 33, 28, 36, 44], 'BidPrice': [17660.0, 17659.0, 17658.0, 17657.0, 17656.0], 'BidVolSum': 125, 'BidVolume': [9, 25, 28, 30, 33], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [0, 0, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [-1, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17663.0, 'FirstDerivedAskVolume': 2, 'FirstDerivedBidPrice': 17657.0, 'FirstDerivedBidVolume': 10, 'TargetKindPrice': 17677.08, 'Time': '12:03:52.522000'}
Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17661.0, 17662.0, 17663.0, 17664.0, 17665.0], 'AskVolSum': 153, 'AskVolume': [12, 33, 28, 36, 44], 'BidPrice': [17660.0, 17659.0, 17658.0, 17657.0, 17656.0], 'BidVolSum': 126, 'BidVolume': [10, 25, 28, 30, 33], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [0, 0, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [1, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17663.0, 'F

Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17661.0, 17662.0, 17663.0, 17664.0, 17665.0], 'AskVolSum': 157, 'AskVolume': [16, 34, 27, 36, 44], 'BidPrice': [17660.0, 17659.0, 17658.0, 17657.0, 17656.0], 'BidVolSum': 121, 'BidVolume': [5, 24, 28, 31, 33], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [1, 0, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [-4, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17663.0, 'FirstDerivedAskVolume': 2, 'FirstDerivedBidPrice': 17657.0, 'FirstDerivedBidVolume': 10, 'TargetKindPrice': 17677.07, 'Time': '12:03:55.397000'}
Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17661.0, 17662.0, 17663.0, 17664.0, 17665.0], 'AskVolSum': 157, 'AskVolume': [16, 34, 27, 36, 44], 'BidPrice': [17660.0, 17659.0, 17658.0, 17657.0, 17656.0], 'BidVolSum': 121, 'BidVolume': [5, 24, 28, 31, 33], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [0, 0, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [0, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17663.0, 'Fi

Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17663.0, 17664.0, 17665.0, 17666.0, 17667.0], 'AskVolSum': 228, 'AskVolume': [12, 36, 44, 61, 75], 'BidPrice': [17662.0, 17661.0, 17660.0, 17659.0, 17658.0], 'BidVolSum': 101, 'BidVolume': [10, 21, 24, 24, 22], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [0, 0, 0, 1, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [-2, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17665.0, 'FirstDerivedAskVolume': 2, 'FirstDerivedBidPrice': 17658.0, 'FirstDerivedBidVolume': 3, 'TargetKindPrice': 17677.07, 'Time': '12:03:58.647000'}
Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17663.0, 17664.0, 17665.0, 17666.0, 17667.0], 'AskVolSum': 228, 'AskVolume': [12, 36, 44, 61, 75], 'BidPrice': [17662.0, 17661.0, 17660.0, 17659.0, 17658.0], 'BidVolSum': 97, 'BidVolume': [9, 17, 24, 24, 23], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [0, 0, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [-1, -4, 0, 0, 1], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17665.0, 'F

...

Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17663.0, 17664.0, 17665.0, 17666.0, 17667.0], 'AskVolSum': 232, 'AskVolume': [19, 37, 42, 62, 72], 'BidPrice': [17661.0, 17660.0, 17659.0, 17658.0, 17657.0], 'BidVolSum': 110, 'BidVolume': [11, 26, 24, 23, 26], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [-3, 0, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [5, 9, 0, 0, -4], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17665.0, 'FirstDerivedAskVolume': 10, 'FirstDerivedBidPrice': 17658.0, 'FirstDerivedBidVolume': 11, 'TargetKindPrice': 17668.04, 'Time': '12:04:03.647000'}
Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17662.0, 17663.0, 17664.0, 17665.0, 17666.0], 'AskVolSum': 163, 'AskVolume': [1, 21, 37, 42, 62], 'BidPrice': [17661.0, 17660.0, 17659.0, 17658.0, 17657.0], 'BidVolSum': 109, 'BidVolume': [10, 26, 24, 23, 26], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [-7, 2, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [-1, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17665.0

Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17662.0, 17663.0, 17664.0, 17665.0, 17666.0], 'AskVolSum': 167, 'AskVolume': [6, 20, 37, 42, 62], 'BidPrice': [17661.0, 17660.0, 17659.0, 17658.0, 17657.0], 'BidVolSum': 99, 'BidVolume': [6, 15, 25, 23, 30], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [0, 0, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [-2, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17665.0, 'FirstDerivedAskVolume': 10, 'FirstDerivedBidPrice': 17658.0, 'FirstDerivedBidVolume': 3, 'TargetKindPrice': 17671.53, 'Time': '12:04:06.022000'}
Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17662.0, 17663.0, 17664.0, 17665.0, 17666.0], 'AskVolSum': 168, 'AskVolume': [6, 20, 37, 42, 63], 'BidPrice': [17661.0, 17660.0, 17659.0, 17658.0, 17657.0], 'BidVolSum': 99, 'BidVolume': [6, 15, 25, 23, 30], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [0, 0, 0, 0, 1], 'DiffAskVolSum': 0, 'DiffBidVol': [0, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17665.0, 'FirstD

Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17661.0, 17662.0, 17663.0, 17664.0, 17665.0], 'AskVolSum': 157, 'AskVolume': [18, 37, 23, 37, 42], 'BidPrice': [17660.0, 17659.0, 17658.0, 17657.0, 17656.0], 'BidVolSum': 105, 'BidVolume': [2, 18, 23, 30, 32], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [0, 0, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [0, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17664.0, 'FirstDerivedAskVolume': 8, 'FirstDerivedBidPrice': 17657.0, 'FirstDerivedBidVolume': 3, 'TargetKindPrice': 17671.53, 'Time': '12:04:08.522000'}
Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17662.0, 17663.0, 17664.0, 17665.0, 17666.0], 'AskVolSum': 202, 'AskVolume': [37, 23, 37, 42, 63], 'BidPrice': [17660.0, 17659.0, 17658.0, 17657.0, 17656.0], 'BidVolSum': 113, 'BidVolume': [10, 18, 23, 30, 32], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [0, 0, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [8, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17664.0, 'Fir

Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17662.0, 17663.0, 17664.0, 17665.0, 17666.0], 'AskVolSum': 194, 'AskVolume': [17, 24, 37, 43, 73], 'BidPrice': [17661.0, 17660.0, 17659.0, 17658.0, 17657.0], 'BidVolSum': 93, 'BidVolume': [1, 13, 23, 25, 31], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [2, 1, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [0, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17665.0, 'FirstDerivedAskVolume': 18, 'FirstDerivedBidPrice': 17657.0, 'FirstDerivedBidVolume': 11, 'TargetKindPrice': 17679.43, 'Time': '12:04:10.647000'}
Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17662.0, 17663.0, 17664.0, 17665.0, 17666.0], 'AskVolSum': 194, 'AskVolume': [17, 24, 37, 43, 73], 'BidPrice': [17661.0, 17660.0, 17659.0, 17658.0, 17657.0], 'BidVolSum': 93, 'BidVolume': [1, 13, 23, 25, 31], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [0, 0, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [0, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17665.0, 'Firs

Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17661.0, 17662.0, 17663.0, 17664.0, 17665.0], 'AskVolSum': 127, 'AskVolume': [2, 15, 30, 37, 43], 'BidPrice': [17660.0, 17659.0, 17658.0, 17657.0, 17656.0], 'BidVolSum': 123, 'BidVolume': [12, 23, 25, 31, 32], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [0, 0, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [-1, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17663.0, 'FirstDerivedAskVolume': 1, 'FirstDerivedBidPrice': 17657.0, 'FirstDerivedBidVolume': 11, 'TargetKindPrice': 17679.43, 'Time': '12:04:12.897000'}
Topic: Q/TFE/MXFL1, Quote: {'AskPrice': [17661.0, 17662.0, 17663.0, 17664.0, 17665.0], 'AskVolSum': 127, 'AskVolume': [2, 15, 30, 37, 43], 'BidPrice': [17660.0, 17659.0, 17658.0, 17657.0, 17656.0], 'BidVolSum': 123, 'BidVolume': [12, 23, 25, 31, 32], 'Code': 'MXFL1', 'Date': '2021/12/07', 'DiffAskVol': [0, 0, 0, 0, 0], 'DiffAskVolSum': 0, 'DiffBidVol': [0, 0, 0, 0, 0], 'DiffBidVolSum': 0, 'FirstDerivedAskPrice': 17663.0, 'Fi

In [34]:
api.quote.unsubscribe(contract_txf, quote_type=sj.constant.QuoteType.BidAsk)

Response Code: 200 | Event Code: 16 | Info: Q/*/MXFL1 | Event: Subscribe or Unsubscribe ok


練習用: snapshots

In [52]:
contracts = [api.Contracts.Stocks['2330'], api.Contracts.Stocks['3481']]
snapshots = api.snapshots(contracts)
snapshots

[Snapshot(ts=1638879202629000000, code='2330', exchange='TSE', open=602.0, high=603.0, low=597.0, close=599.0, tick_type=<TickType.Buy: 'Buy'>, change_price=-1.0, change_rate=-0.17, change_type=<ChangeType.Down: 'Down'>, average_price=599.66, volume=1, total_volume=10275, amount=599000, total_amount=6161459000, yesterday_volume=23589.0, buy_price=598.0, buy_volume=535.0, sell_price=599.0, sell_volume=190, volume_ratio=0.44),
 Snapshot(ts=1638879226038000000, code='3481', exchange='TSE', open=18.5, high=19.0, low=18.4, close=18.9, tick_type=<TickType.Sell: 'Sell'>, change_price=0.7, change_rate=3.85, change_type=<ChangeType.Up: 'Up'>, average_price=18.72, volume=3, total_volume=216389, amount=56700, total_amount=4049845850, yesterday_volume=138924.0, buy_price=18.9, buy_volume=1579.0, sell_price=18.95, sell_volume=3496, volume_ratio=1.56)]